In [39]:
import numpy as np
import pandas as pd
import sys
import gc
import joblib
import pathlib
import json
import glob
from tqdm.notebook import tqdm

# visualize
import matplotlib.pyplot as plt
import matplotlib.style as style
import seaborn as sns
from matplotlib import pyplot
from matplotlib.ticker import ScalarFormatter
sns.set_context("talk")
style.use('seaborn-colorblind')

import plotly.graph_objects as go

In [40]:
df = pd.read_csv("/Users/sandeep/Workspace/APEX/RecoveryStocksArbitrage/Yash_Recovery_Stocks.csv")
df = df.drop(index = 72)
df

,Ticker,Name,Total Return:M-6,Rev - 1 Yr Gr:Q,Market Cap,Market Cap.1,Price:D-1,P/E
0,BASE US Equity,COUCHBASE INC,-30.70,20.16,9.150515e+08,9.150515e+08,20.99,NaN
1,SPT US Equity,SPROUT SOCIAL INC - CLASS A,-30.92,45.82,3.414239e+09,3.414239e+09,63.22,NaN
2,ASAN US Equity,ASANA INC - CL A,-31.53,70.34,8.948612e+09,8.948612e+09,48.00,NaN
3,XM US Equity,QUALTRICS INTERNATIONAL-CL A,-32.03,47.98,1.543719e+10,1.543719e+10,27.06,NaN
4,RPAY US Equity,REPAY HOLDINGS CORP,-32.19,62.42,1.552755e+09,1.552755e+09,17.15,NaN
...,...,...,...,...,...,...,...,...
67,API US Equity,AGORA INC-ADR,-70.62,46.00,1.083391e+09,1.083391e+09,9.35,NaN
68,KC US Equity,KINGSOFT CLOUD HOLDINGS-ADR,-74.03,39.62,1.781101e+09,1.781101e+09,7.37,NaN
69,STNE US Equity,STONECO LTD-A,-74.32,57.30,4.528934e+09,4.528934e+09,14.66,19.15
70,TUYA US Equity,TUYA INC,-75.15,44.85,2.783783e+09,2.783783e+09,4.96,NaN


In [49]:
df['Ticker'] = df['Ticker'].replace("", "US Equity")
df = df.replace(to_replace=r' US Equity', value='', regex=True)
df

,Ticker,Name,Total Return:M-6,Rev - 1 Yr Gr:Q,Market Cap,Market Cap.1,Price:D-1,P/E
0,BASE,COUCHBASE INC,-30.70,20.16,9.150515e+08,9.150515e+08,20.99,NaN
1,SPT,SPROUT SOCIAL INC - CLASS A,-30.92,45.82,3.414239e+09,3.414239e+09,63.22,NaN
2,ASAN,ASANA INC - CL A,-31.53,70.34,8.948612e+09,8.948612e+09,48.00,NaN
3,XM,QUALTRICS INTERNATIONAL-CL A,-32.03,47.98,1.543719e+10,1.543719e+10,27.06,NaN
4,RPAY,REPAY HOLDINGS CORP,-32.19,62.42,1.552755e+09,1.552755e+09,17.15,NaN
...,...,...,...,...,...,...,...,...
67,API,AGORA INC-ADR,-70.62,46.00,1.083391e+09,1.083391e+09,9.35,NaN
68,KC,KINGSOFT CLOUD HOLDINGS-ADR,-74.03,39.62,1.781101e+09,1.781101e+09,7.37,NaN
69,STNE,STONECO LTD-A,-74.32,57.30,4.528934e+09,4.528934e+09,14.66,19.15
70,TUYA,TUYA INC,-75.15,44.85,2.783783e+09,2.783783e+09,4.96,NaN


In [99]:
import yfinance as yf

# Data gathering
historical_data = dict()

# map ticker labels to their corresponding 3mo historical data
for ticker in df['Ticker']:
    ticker = ticker.replace(" ","")
    #Lets look at the last 3m of data
    data = yf.Ticker(ticker).history(period = '3mo')
    # Add day of the week so we can start to see weekly patterns
    data = data.reset_index(0)
    data['Dayofweek'] = pd.to_datetime(data['Date']).dt.dayofweek
    data = data.drop(columns=['High','Low','Dividends', 'Stock Splits'])
    data['Intraday'] = data['Close'] / data['Open']
    data = data.reindex(columns = ['Date', 'Dayofweek', 'Open', 'Close', 'Volume', 'Intraday'])
    historical_data[ticker] = data


In [103]:
for ticker, data in historical_data.items():
    print(f'Current Ticker: {ticker}')

Current Ticker: BASE
Current Ticker: SPT
Current Ticker: ASAN
Current Ticker: XM
Current Ticker: RPAY
Current Ticker: VEEV
Current Ticker: NCNO
Current Ticker: CLVT
Current Ticker: VERI
Current Ticker: OLED
Current Ticker: CISO
Current Ticker: CRWD
Current Ticker: BSY
Current Ticker: APPS
Current Ticker: ZEN
Current Ticker: EVCM
Current Ticker: NLST
Current Ticker: FIVN
Current Ticker: RNG
Current Ticker: PCOR
Current Ticker: AVLR
Current Ticker: ESTC
Current Ticker: INTA
Current Ticker: QTWO
Current Ticker: SQSP
Current Ticker: COUP
Current Ticker: WKME
Current Ticker: PLTR
Current Ticker: CXM
Current Ticker: FROG
Current Ticker: PAR
Current Ticker: VRNS
Current Ticker: CLBT
Current Ticker: DCT
Current Ticker: CAN
Current Ticker: FOUR
Current Ticker: PATH
Current Ticker: PAYA
Current Ticker: PRCH
Current Ticker: DOMO
Current Ticker: FSLY
Current Ticker: HCAT
Current Ticker: AMEH
Current Ticker: TWLO
Current Ticker: SGFY
Current Ticker: CDLX
Current Ticker: ALKT
Current Ticker: LPRO
Cu

         Date  Dayofweek        Open       Close   Volume  Intraday
0  2021-11-01          0  137.990005  135.279999  1492000  0.980361
1  2021-11-02          1  134.610001  128.190002  1801700  0.952307
2  2021-11-03          2  128.979996  132.440002  2257700  1.026826
3  2021-11-04          3  126.720001  131.669998  1834700  1.039062
4  2021-11-05          4  131.470001  128.070007  2102200  0.974139
..        ...        ...         ...         ...      ...       ...
58 2022-01-25          1   52.689999   50.570000  2972300  0.959765
59 2022-01-26          2   54.000000   48.139999  4594700  0.891481
60 2022-01-27          3   49.099998   45.459999  5250800  0.925866
61 2022-01-28          4   46.419998   48.000000  5291300  1.034037
62 2022-01-31          0   48.549999   51.790001  2738213  1.066735

[63 rows x 6 columns]
